In [22]:
import urllib.request
import re
import html
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [23]:
# Собираем ссылки, которые ведут на сраницы с новостями на сайте этой газеты.


def get_urls():
    url = 'http://mk.tula.ru/news/%s/?PAGEN_1=%s'
    topics = ['health', 'culture', 'society', 'incedent', 'sport',
              'business', 'oboronka']
    k = 1
    i = 1
    links_list = []
    for t in topics:
        i = 1
        k = 1
        # В разделе "Оборонка" всего 6 страниц выдачи со стьатями.
        # Исключив этот раздел, k можно поставить больше.
        # Я выкачивала с 4-х страниц каждого раздела - 840 статей.
        # for k in range(1, 7):
        for k in range(1, 5):
            i = 1
            while url and i < 2:
                u = url % (t, k)
            # print(u)
                w = urlopen(u, timeout=5)
                h = w.read()
                s = BeautifulSoup(h, 'html5lib')
                i += 1
                if True:
                    links = s.find_all('a')
                    for l in links:
                        l.get('href')
                        if l.get('href').startswith('/news/n/'):
                            if l.get('href') not in links_list:
                                links_list.append(l.get('href'))
    return links_list


In [24]:
# Создаёем CSV-таблицу и вносим в неё заголовки столбцов


def csv_file():
    if not os.path.exists('newspaper/'):
        os.makedirs('newspaper')
    p_csv = 'newspaper/CSV-table.csv'
    headers = 'path\\tаuthor\\theader\\tcreated\\tsphere\\ttopic\\tstyle\\t \
               audience_age\\taudience_level\\taudience_size\\tsource\\t \
               publication\\t publ_year\\tmedium\\tcountry\\tregion\\tlanguage'
    with open(p_csv, 'w', encoding="utf-8") as f:
        f.write(headers + '\n')


In [25]:
# Этой функцией мы будем считать количество статей в папке,
# чтобы присваивать им соответствующий номер


def num_files(path, a, b):
    list_dir = []
    list_dir = os.listdir(path)
    num = 0
    for file in list_dir:
        if file.endswith(a) or file.endswith(b):
            num += 1
    return num


In [26]:
# Извлекаем из html-кода страницы метаднные и текст статьи


def csv_data(page_html):
    regAuthor = re.compile('<p><span>Автор:</span>&nbsp;(.*?)</p>',
                           flags=re.DOTALL)
    authors = regAuthor.findall(page_html)
    if len(authors) == 0:
        au = 'None'
    else:
        au = authors[0]
    regTitle = re.compile('<title>(.*?)</title>')
    title = regTitle.findall(page_html)[0]
    regDate = re.compile('<div class="data_news">.*?<span>(.*?)</span>',
                         flags=re.DOTALL)
    date_time = regDate.findall(page_html)[0]
    date = date_time.split()[0]  # отделяем дату от времени
    month = date.split('.')[1]  # берём месяц
    year = date.split('.')[2]  # берём год
    regCat = re.compile('<div class="poster_content_img">.*?<span>(.\\D*?)</span>.*?<img',
                        flags=re.DOTALL)
    category = regCat.findall(page_html)
    if len(category) == 0:
        cat = 'None'
    else:
        cat = category[0]
    regHTMLt = re.compile('<div style="clear:right; "></div>(.*?)<div',
                          flags=re.DOTALL)
    HTMLt = regHTMLt.findall(page_html)
    return au, title, date, month, year, cat, HTMLt


In [27]:
# Создаём папки и присваиваем файлу со статьёй имя с нужным номером


def create_dirs(month, year):
    if not os.path.exists('newspaper/plain/%s/%s/' % (year, month)):
        os.makedirs('newspaper/plain/%s/%s' % (year, month))
    p = 'newspaper/plain/%s/%s/' % (year, month)
    n = num_files(p, '.txt', '.xml') + 1
    filename = 'статья%s.txt' % n
    my_path = p + filename
    if not os.path.exists('newspaper/mystem-plain/%s/%s/' % (year, month)):
        os.makedirs('newspaper/mystem-plain/%s/%s' % (year, month))
    if not os.path.exists('newspaper/mystem-xml/%s/%s/' % (year, month)):
        os.makedirs('newspaper/mystem-xml/%s/%s' % (year, month))
    return my_path, filename


In [28]:
# Пишем в файл пустой текст, размечаем его с помощью Mystem.
# Добавляем метаданные в csv-таблицу.
# Переписываем файл с пустым текстом, добавляя в него строки с метаданными.
# Наверное, это можно было бы разделить на отдельны функции.


def create_files(my_path, au, title, date, month, year, cat, HTMLt,
                 pageUrl, filename):
    # Печатаем в файл текст статьи
    with open(my_path, 'w', encoding="utf-8") as f:
        for element in HTMLt:
            regTag = re.compile('<.*?>')
            element_cl = regTag.sub('', element)
            f.write( html.unescape(element_cl) )
        f.close()
    # print(my_path)
    # Добавляем метаданные в CSV-таблицу
    row = '%s\\t%s\\t%s\\t%s\\tпублицистика\\t%s\\tнейтральный\\tн-возраст\\t \
            н-уровень\\tрегиональная\\t%s\\tМолодой коммунар\\t%s\\t \
            газета\\tРоссия\\tТула\\tru'
    p_csv = 'newspaper/CSV-table.csv'
    with open(p_csv, 'a', encoding="utf-8") as f:
        f.write(row % (my_path, au, title, date, cat, pageUrl, year) + '\n')
        f.close
    # Mystem
    filename_xml = filename.split('.')[0] + '.xml'
    os.system("/applications/mystem -lcgid --eng-gr \
                newspaper/plain/%s/%s/%s newspaper/mystem-plain/%s/%s/%s" %
              (year, month, filename, year, month, filename))
    os.system("/applications/mystem -lcgid --eng-gr --format xml \
                newspaper/plain/%s/%s/%s newspaper/mystem-xml/%s/%s/%s" %
              (year, month, filename, year, month, filename_xml))
    # Переписываем первый файл, добавляя строки с метаданными.
    with open(my_path, 'w', encoding="utf-8") as f:
        f.write('@au ' + au + '\n')
        f.write('@ti ' + title + '\n')
        f.write('@da ' + date + '\n')
        f.write('@topic ' + cat + '\n')
        f.write('@url ' + pageUrl + '\n')
        for element in HTMLt:
            regTag = re.compile('<.*?>')
            element_cl = regTag.sub('', element)
            f.write( html.unescape(element_cl) )
        f.close()
    with open('newspaper/texts.txt', 'a', encoding="utf-8") as f:
        for element in HTMLt:
            regTag = re.compile('<.*?>')
            element_cl = regTag.sub('', element)
            f.write( html.unescape(element_cl) )
        f.close()
            

In [29]:
# Собственно функция, извлекающая html-код всей страницы
# (потом извлеченный код используется в функциях выше)


def download_page(pageUrl):
    try:
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
    except:
        print('Error at', pageUrl)
        text = ''
    return text


In [30]:
# Применяем все вышенаписанные функции для каждой полученной ссылке


def news_pages(links_list):
    commonUrl = 'http://mk.tula.ru'
    for link in links_list:
        pageUrl = commonUrl + link
        page_html = str(download_page(pageUrl))
        au_csv, title_csv, date_csv, month_csv, year_csv, \
        cat_csv, HTMLtext = csv_data(page_html)
        path_csv, f_name = create_dirs(month_csv, year_csv)
        words_number = create_files(path_csv, au_csv, title_csv, date_csv, month_csv,
                     year_csv, cat_csv, HTMLtext, pageUrl, f_name)
        break


In [31]:
def main():
    print('Начинаю работать.')
    csv_file()
    all_links = get_urls()
    news_pages(all_links)
    print('Всё готово.')

main()


Начинаю работать.
Всё готово.
